# Сбор текстов с сайта Сравни.ру

    Для реализации большинства алгоритмов анализа тональности необходим обучающий массив данных (набор отзывов, для которых известна эмоциональная окраска). Так как на сегодняшний день существует очень мало публичных русскоязычных коллекций отзывов, которые можно было бы использовать для решения задачи классификации текстов, и не обнаружено ни одной русскоязычной публичной коллекции отзывов о банках, было принято решение подготовить собственный корпус текстов.
    
    На сайте Сравни.ру оценки пользователей напрямую связаны с тональностью текста: оценка «5» соответствует только отзывам, содержащим описание положительных характеристик банка, продукта или услуги или выражение благодарности сотрудникам банка, поэтому данный веб-сайт был выбран для составления корпуса.
    
 

### Импорт библиотек

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

### Сбор корпуса

    Изучив структуру сайта Сравни.ру, обнаружили, что полный текст каждого отдельного отзыва находится в соответствующем ему HTML-файле, поэтому сначала необходимо собрать все ссылки на необходимые файлы. 

In [2]:
def links(page_number):
    # для нашего исследования было важно оставить возможность подставлять в код отдельные страницы и оценки
    url = f'https://www.sravni.ru/banki/otzyvy?page={page_number}&rated=five&reviewObjectId=' 
    response = requests.get(url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
    soup = BeautifulSoup(response.text, 'html')
    quotes = soup.find_all('span', class_='sc-1fxln1u-27 cPAIqy')
    
    return quotes

In [3]:
data = []
for page_number in range(130, 160):
    urls = links(page_number=page_number)
    data.append(urls)

In [4]:
data = " ".join(map(str,data))
print(data)

[<span class="sc-1fxln1u-27 cPAIqy"><a href="/bank/tinkoff-bank/otzyvy/526561"><div class="sc-1fxln1u-19 cNhSll">Отличная работа службы поддержки клиентов</div></a></span>, <span class="sc-1fxln1u-27 cPAIqy"><a href="/bank/tinkoff-bank/otzyvy/526559"><div class="sc-1fxln1u-19 cNhSll">Как я забирал дебетовую карту</div></a></span>, <span class="sc-1fxln1u-27 cPAIqy"><a href="/bank/tinkoff-bank/otzyvy/526557"><div class="sc-1fxln1u-19 cNhSll">Переход на валютные платежи с контрагентом</div></a></span>, <span class="sc-1fxln1u-27 cPAIqy"><a href="/bank/tinkoff-bank/otzyvy/526555"><div class="sc-1fxln1u-19 cNhSll">Вопросы по обслуживанию: заказ справок, карт и пр.</div></a></span>, <span class="sc-1fxln1u-27 cPAIqy"><a href="/bank/tinkoff-bank/otzyvy/526554"><div class="sc-1fxln1u-19 cNhSll">Обслуживание клиентов в приложении</div></a></span>, <span class="sc-1fxln1u-27 cPAIqy"><a href="/bank/tinkoff-bank/otzyvy/526553"><div class="sc-1fxln1u-19 cNhSll">Мое мнение об этом банке крайне поло

    Регулярным выражением удаляем из списка названия тегов для дальнейшей работы.


In [5]:
result = re.findall(r'/bank/\w+/otzyvy/......', data)
print(result)

['/bank/sovkombank/otzyvy/526542', '/bank/sovkombank/otzyvy/526536', '/bank/sovkombank/otzyvy/526535', '/bank/sovkombank/otzyvy/526524', '/bank/sovkombank/otzyvy/526518', '/bank/sovkombank/otzyvy/526517', '/bank/sovkombank/otzyvy/526515', '/bank/sovkombank/otzyvy/526507', '/bank/sovkombank/otzyvy/526503', '/bank/sovkombank/otzyvy/526496', '/bank/sovkombank/otzyvy/526492', '/bank/sovkombank/otzyvy/526484', '/bank/sovkombank/otzyvy/526475', '/bank/sovkombank/otzyvy/526474', '/bank/sovkombank/otzyvy/526471', '/bank/sovkombank/otzyvy/526447', '/bank/sovkombank/otzyvy/526444', '/bank/sovkombank/otzyvy/526443', '/bank/sovkombank/otzyvy/526438', '/bank/sovkombank/otzyvy/526437', '/bank/sovkombank/otzyvy/526428', '/bank/sovkombank/otzyvy/526427', '/bank/sovkombank/otzyvy/526421', '/bank/sovkombank/otzyvy/526418', '/bank/sovkombank/otzyvy/526415', '/bank/sovkombank/otzyvy/526399', '/bank/sovkombank/otzyvy/526386', '/bank/sovkombank/otzyvy/526384', '/bank/sovkombank/otzyvy/526382', '/bank/sovkom

In [6]:
column_values = pd.Series(result)

    Далее, тем же методом с кода страниц, находящихся по каждой из полученных ссылок, сохраняем сам текст отзыва  и соответствующие ему метаданные, а именно: дату написания отзыва, имя пользователя, тему обращения и название банка. 

In [7]:
text = []
date = []
name = []
theme = []
for i in column_values:
    url = ('https://www.sravni.ru'+i)
    response = requests.get(url, headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'})
    soup = BeautifulSoup(response.text, 'lxml')
    quotes = soup.find_all('div', class_='sc-lpzn2g-13 eCTrNq')
    d = soup.find_all('span', class_='sc-lpzn2g-8 hgjOYK')
    c = soup.find_all('span', class_='sc-lpzn2g-6 nXSPo')
    n = soup.find_all('span', class_='sc-lpzn2g-5 hgMCyH')
    t = soup.find_all('div', class_='sc-lpzn2g-15 bhGCHf')
    
    text.append(quotes)
    date.append(d)
    name.append(n)
    theme.append(t)
bank = []
for i in column_values:
    bank.append(i.split('/')[2])

    Собранные данные очищаем с помощью регулярных выражений и на основе них формируем датафрейм.

In [8]:
n = []
for i in range(0, len(name)):
    string = ' '.join(re.findall(r'[А-Яа-я]+', str(name[i])))
    string = re.sub('</span>','', str(string))
    string = string.capitalize()
    n.append(string)

In [9]:
d = []
for i in range(0, len(date)):
    string = ' '.join(re.findall(r'...[А-Яа-я]+......', str(date[i])))
    string = re.sub('</span>','', str(string))
    d.append(string)

In [10]:
u = []
for i in range(0, len(text)):
    string = ' '.join(re.findall(r'[А-Яа-я].+', str(text[i])))
    string = re.sub('</p>','', str(string))
    
    u.append(string)

In [11]:
t = []
for i in range(0, len(theme)):
    string = ' '.join(re.findall(r'[А-Яа-я]+', str(theme[i])))
    string = re.sub('</span>','', str(string))
    t.append(string)


In [12]:
u = {"text":u}
df = pd.DataFrame(u)
df['rated'] = 1
df['coloring'] = 'bad'
df['date'] = d
df['bank'] = bank
df['theme'] = t

In [13]:
nn = []
for i in n:
    i = i.split(' ')
    nn.append(i[0])

    Для возможности проведения большего количества статистических исследований на основе корпуса было принято решение добавить колонку с полом пользователя. Для определения пола пользователя мы сохранили списки женских и мужских имен с сайта Википедия. По наличию каждого имени датафрейма в этих списках проставляем теги ‘female’ для женских имен, ‘male’ для мужских и ‘unkn’ для пользователей, не указавших имя.

In [14]:
with open('fem.txt', 'r', encoding='utf-8') as f:
        fem = f.read()
fem = re.findall(r'[А-Яа-я]+', str(fem))
with open('male.txt', 'r', encoding='utf-8') as m:
        male = m.read()
male = re.findall(r'[А-Яа-я]+', str(male))

In [15]:
df['name'] = nn

In [16]:
def names(name):   
    if name in fem or name == 'Ирина' or name == 'Ольга' or name == 'Надя':
        return 'female'
    elif name in male or name =='Слава':
        return 'male'
    else:
        return 'unkn'
df['sex'] = df['name'].apply(names)    

In [17]:
df = df.drop_duplicates()

In [18]:
df.to_csv('test_five.csv') 